In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/dirty_data.csv')
df.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


In [3]:
df.tail()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
760,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
761,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
762,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
763,2018-12-31T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
764,2018-12-31T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN


In [19]:
df.describe(include='all').T

/home/cairo/anaconda3/envs/book_env/lib/python3.12/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/cairo/anaconda3/envs/book_env/lib/python3.12/site-packages/numpy/lib/function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,765,324,2018-07-05T00:00:00,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
station,765,2,GHCND:USC00280907,398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PRCP,765.0,NaN,NaN,NaN,5.360392,10.002138,0.0,0.0,0.0,5.8,61.7
SNOW,577.0,NaN,NaN,NaN,4.202773,25.086077,0.0,0.0,0.0,0.0,229.0
SNWD,577.0,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,inf
TMAX,765.0,NaN,NaN,NaN,2649.175294,2744.156281,-11.7,13.3,32.8,5505.0,5505.0
TMIN,765.0,NaN,NaN,NaN,-15.914379,24.242849,-40.0,-40.0,-11.1,6.7,23.9
TOBS,398.0,NaN,NaN,NaN,8.632161,9.815054,-16.1,0.15,8.3,18.3,26.1
WESF,11.0,NaN,NaN,NaN,16.290909,9.489832,1.8,8.6,19.3,24.9,28.7
inclement_weather,408,2,False,384,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


In [6]:
df.isnull().sum()

date                   0
station                0
PRCP                   0
SNOW                 188
SNWD                 188
TMAX                   0
TMIN                   0
TOBS                 367
WESF                 754
inclement_weather    357
dtype: int64

In [7]:
df.SNWD.unique()

array([-inf,  inf,  nan])

In [8]:
import numpy as np

df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

577

We will use a dictionary comprehension to return the number of infinite values per column in our dataframe

In [9]:
def get_inf_count(df):
    """Find the number of inf/-inf values per column"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])
                ].shape[0] for col in df.columns
    }

In [10]:
get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

we should look at the summary statistics for snowfall (SNOW), which forms a big part of determining
the snow depth (SNWD). 

We can make a dataframe with two series, where one
contains the summary statistics for the snowfall column when the snow depth is np.inf,
and the other for when it is -np.inf. In addition, we will use the T attribute to transpose
the data for easier viewing

In [11]:
pd.DataFrame({
    'np.inf Snow Depth':
        df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth':
        df[df.SNWD == -np.inf].SNOW.describe()
}).T

,count,mean,std,min,25%,50%,75%,max
np.inf Snow Depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf Snow Depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


We can consider that not having snow could be NaN or 0, but that's not the same for positives inf.

In [12]:
df.describe(include='object')

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


In [13]:
df.head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


Station have 2 unique values, one of them is ?.
Maybe the data was put accidentaly or the software have a problem, we don't know.

### Duplicates

In [14]:
df[df.duplicated()].shape[0]

284

- by default, the keep is 'first', and, for each row that is present more than once, we will get only the additional rows (besides the first)
-  if we pass in keep=False, we will get all the rows that are present more than once

In [15]:
df[df.duplicated(keep=False)].shape[0]

482

In [16]:
df[df.duplicated(['date', 'station'])].shape[0]

284

In [17]:
df[df.duplicated()].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


Remember that the default behavior of duplicated() is to not show the first occurrence, which means that rows 1 and 2 have another matching value in the data
(same for rows 5 and 6

### Duplicated data
Since we know that both stations will be for New York City, we may decide to drop the station column—they may have just been collecting different data.

In [18]:
df[df.WESF.notna()].station.unique()

array(['?'], dtype=object)

How to solve it?
1. Perform type conversion on the date column
2. Save the WESF column as a series
3. Sort the dataframe by the station column in descending order to put the station with no ID (?) last
4. Remove rows that are duplicated based on the date, keeping the first occurrences
5. Drop the station column and set the index to the date column (so that it matches the WESF data)
6. Update the WESF column using the `combine_first()` method to coalesce (just as in SQL for those coming from a SQL background) the values to the first non-null entry

In [20]:
# 1 step
df.date = pd.to_datetime(df.date)

In [21]:
# step 2
station_qm_wesf = df[df.station == '?']\
    .drop_duplicates('date').set_index('date').WESF

In [22]:
station_qm_wesf

date
2018-01-01     NaN
2018-01-04    19.3
2018-01-05     NaN
2018-01-07     NaN
2018-01-08     NaN
              ... 
2018-12-27     NaN
2018-12-28     NaN
2018-12-29     NaN
2018-12-30     NaN
2018-12-31     NaN
Name: WESF, Length: 232, dtype: float64

In [23]:
# step 3
df.sort_values(
    'station', ascending=False, inplace=True
)

In [24]:
# step 4
df_deduped = df.drop_duplicates('date')

In [25]:
# step 5
df_deduped = df_deduped.drop(columns='station')\
    .set_index('date').sort_index()

In [27]:
# step 6
df_deduped = df_deduped.assign(
    WESF = lambda x: x.WESF.combine_first(station_qm_wesf)
)

In [28]:
df_deduped.shape

(324, 8)

In [29]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


We are now left with 324 rows—one for each unique date in our data. We were able to save the WESF column by putting it alongside the data from the other station

### Null data

We can:
- drop it
- replace it with arbitrary value
- impute it using surrounding data 

In [30]:
# To drop all the rows with any null data
df_deduped.dropna().shape

(4, 8)

In [31]:
# Drop only a row if all the columns are null
df_deduped.dropna(how='all').shape # default is 'any'

(324, 8)

Say we wanted to look at snow data; we would most likely want to make sure that our data had values
for SNOW, SNWD, and inclement_weather.

In [32]:
# use a subset of columns to determine what to drop
df_deduped.dropna(
        how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

(293, 8)

Could we performe it along the columns? Yes!
we can provide a threshold for the number of null values that must be observed to drop the data
with the `thresh` argument. 

In [38]:
df_deduped.dropna(
    axis='columns',
    thresh=df_deduped.shape[0] * .75 # 75% of rows
).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

Since we have a lot of null values, we will likely be more interested in keeping these values,
and perhaps finding a better way to represent them

To fill in null values with other data, we use the `fillna()` method, which gives us
the option of specifying a value or a strategy for how to perform the filling. 

In [39]:
df_deduped.loc[:, 'WESF'].fillna(0, inplace=True)
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


That being said, let's try to address some of the remaining issues with the temperature data.

Why we have 5505 in `TMAX` column? That's the temp of the sun. it must be because there was no measured value

In [42]:
# Replace the suspicius values with NaN
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan)
)

We can fill nan values with `fillna()` method.
- 'ffill' to forward-fill
- 'bfill' to back-fill

In [43]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill')
).head()

/tmp/ipykernel_3893/2746109009.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  TMAX=lambda x: x.TMAX.fillna(method='ffill'),
/tmp/ipykernel_3893/2746109009.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  TMIN=lambda x: x.TMIN.fillna(method='ffill')


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


TMAX and TMIN columns on January 1st and 4th. Both are NaN on the 1st because we don't have data before then to bring forward, but the 4th now has the same values as the 3rd

How to handle with nulls and infinite numbers?

- we can use the `np.nan_to_num()` function; it turns NaN into 0 and inf/-inf into very large positive/negative finite numbers
- making it possible for machine learning models to learn from this data

In [44]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD) 
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


for -np.inf , we may choose to set SNWD to 0 since we saw there was no snowfall on those days. However, we don't know what to do with np.inf, and the large positive numbers, arguably, make
this more confusing to interpret

The `clip()` method makes it
possible to cap values at a specific minimum and/or maximum threshold.

In [45]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


Our last strategy is `imputation`. We replace a missing value with a new value derived from the data, using summary statistics or data from other observations.

We can combine imputation with the `fillna()` method

In [47]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


If we want to run the same calculation on all the **columns**, we should use the `apply()` method instead of `assign()` since it saves us the redundancy of having to write the same calculation for each of the columns. 

let's fill in all the missing values
with the rolling 7-day median of their values, setting the number of periods required for
the calculation to zero to ensure that we don't introduce extra null values

In [51]:
df_deduped.apply(lambda x:
    # Rolling 7-day median (covered in chapter 4).
    # we set min_periods (# of periods required for
    # calculation) to 0 so we always get a result
    x.fillna(x.rolling(7, min_periods=0).median())
    ).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


In [52]:
df_deduped.reindex(
    pd.date_range('2018-01-01', '2018-12-31', freq='D')
).apply(lambda x: x.interpolate().head(10))

/tmp/ipykernel_3893/141895325.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  ).apply(lambda x: x.interpolate().head(10))


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False


Check out January 9th, which we didn't have previously—the values for TMAX, TMIN,
and TOBS are the average of the values for the day prior (January 8th) and the day after
(January 10th)